In [7]:
import pandas as pd
import numpy as np
import glob
import re

We first merge individual CSVs to create a tidy dataset. Our units of interest (comprising one row) consist of unique combinations of state, gender, and year. Since the goal of this project is to look at recidivism, we are interested only in convicted individuals in prison. 

In [25]:
fileList = glob.glob("./Data/*.csv")
dfNames = [re.search("[(\\\)](.*)\.",file).group(1) for file in fileList]

for idx in range(len(dfNames)):
    df = pd.read_csv(fileList[idx])
    name = dfNames[idx]
    vars()[name] = df
    print(vars()['name'])


Age_group
Caste
Death_sentence
Domicile
Education
Education_facilities
Inmates_death
Inmates_escapee
IPC_crime_inmates_convicted
IPC_crime_inmates_under_trial
Jail wise population of prison inmates
Prison_details_2015
Recidivism
Rehabilitation
Religion
Sentence_period
Tranquillity
Vocational_training
Wages


In [80]:
Sentence_period.query('is_state==1'). \
drop(["is_state"],axis=1).pivot_table(index=["state_name","year"], 
                   columns = ["gender","sentence_period"])

age_16_18_years                         \
gender                           Female                          
sentence_period     1 Less than 2 years 10  Less than 3 months   
state_name     year                                              
Andhra Pradesh 2001                 0.0                    NaN   
               2002                 0.0                    NaN   
               2003                 0.0                    NaN   
               2004                 0.0                    NaN   
               2005                 0.0                    NaN   
...                                 ...                    ...   
West Bengal    2009                 0.0                    NaN   
               2010                 0.0                    NaN   
               2011                 0.0                    NaN   
               2012                 0.0                    NaN   
               2013                 0.0                    0.0   

                                                                  \
gender                                                             
sentence_period     10-13 Plus years 2-4 Plus years 3 - 6 months   
state_name     year                                                
Andhra Pradesh 2001              0.0            0.0          0.0   
               2002              0.0            0.0          0.0   
               2003              0.0            0.0          0.0   
               2004              0.0            0.0          0.0   
               2005              0.0            0.0          0.0   
...                              ...            ...          ...   
West Bengal    2009              0.0            0.0          0.0   
               2010              0.0            0.0          0.0   
               2011              0.0            0.0          0.0   
               2012              0.0            0.0          0.0   
               2013              0.0            0.0          NaN   

                                                         \
gender                                                    
sentence_period     3 less than 6 months 5-6 Plus years   
state_name     year                                       
Andhra Pradesh 2001                  NaN            0.0   
               2002                  NaN            0.0   
               2003                  NaN            0.0   
               2004                  NaN            0.0   
               2005                  NaN            0.0   
...                                  ...            ...   
West Bengal    2009                  NaN            0.0   
               2010                  NaN            0.0   
               2011                  NaN            0.0   
               2012                  NaN            0.0   
               2013                  0.0            0.0   

                                                             \
gender                                                        
sentence_period     6 months less than 1 Yr. 7-9 Plus years   
state_name     year                                           
Andhra Pradesh 2001                      0.0            0.0   
               2002                      0.0            0.0   
               2003                      0.0            0.0   
               2004                      0.0            0.0   
               2005                      0.0            0.0   
...                                      ...            ...   
West Bengal    2009                      0.0            0.0   
               2010                      0.0            0.0   
               2011                      0.0            0.0   
               2012                      0.0            0.0   
               2013                      0.0            0.0   

                                        ...     age_50_above                 \
gender                                  ...             Male                  
sentence_period     Ca

## Age_group


In [78]:
Age_group

,state_name,is_state,year,category,type,gender,age_16_18,age_18_30,age_30_50,age_50_above
0,A & N Islands,0,2001,Foreigners,Convicts,Female,0.0,0,0,0
1,A & N Islands,0,2001,Foreigners,Convicts,Male,0.0,12,139,2
2,A & N Islands,0,2001,Foreigners,Detenus,Female,0.0,0,0,0
3,A & N Islands,0,2001,Foreigners,Detenus,Male,0.0,0,0,0
4,A & N Islands,0,2001,Foreigners,Others,Female,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
7147,West Bengal,1,2013,Indians,Detenus,Male,0.0,0,0,0
7148,West Bengal,1,2013,Indians,Others,Female,0.0,0,0,0
7149,West Bengal,1,2013,Indians,Others,Male,0.0,1,0,0
7150,West Bengal,1,2013,Indians,Under Trials,Female,0.0,271,330,125


In [75]:
Age_group.query('type=="Convicts" and is_state==1 and category!="Foreigners"'). \
drop(["type","category","is_state"],axis=1)

,state_name,year,gender,age_16_18,age_18_30,age_30_50,age_50_above
216,Andhra Pradesh,2001,Female,0.0,28,50,24
217,Andhra Pradesh,2001,Male,5.0,1638,2085,388
232,Andhra Pradesh,2002,Female,0.0,44,71,36
233,Andhra Pradesh,2002,Male,3.0,1982,2292,440
248,Andhra Pradesh,2003,Female,0.0,53,73,46
...,...,...,...,...,...,...,...
7113,West Bengal,2011,Male,0.0,1369,2219,956
7128,West Bengal,2012,Female,0.0,94,116,115
7129,West Bengal,2012,Male,0.0,1650,2363,793
7144,West Bengal,2013,Female,0.0,78,112,84


## Caste
One thing to be mindful of: there is implicit missing data on castes among convicts. The number of total convicts we have caste data for is substantially smaller than the number of admitted convicts (a variable in the recidivism dataframe). For now, I'm leaving the caste data values as counts rather than proportions as a result.

In [52]:
Caste = Caste.iloc[:,:6].query('is_state==1'). \
pivot_table(index=["state_name","year"], 
                   columns = ["gender","caste"],
                   values="convicts")
#.df_long.pivot_table(index=["state_name",])

In [53]:
Caste

gender              Female                 Male                  
caste                  OBC Others  SC  ST   OBC Others    SC   ST
state_name     year                                              
Andhra Pradesh 2001     50     12  33   7  1526    939  1236  418
               2002     65     29  34  23  1833   1327  1177  384
               2003    105     31  25  11  2279   1162  1153  381
               2004    116     31  35  15  2146   1204   918  300
               2005    120     19  41  18  2201   1312  1002  298
...                    ...    ...  ..  ..   ...    ...   ...  ...
West Bengal    2009     29    261  70  45   401   3786   838  489
               2010     51    240  67  23   607   3422   818  550
               2011     34    240  59  29   319   3407   751  821
               2012     15    318  49  22   459   3732   982  594
               2013     38    225  87  17   593   3647   847  436

[355 rows x 8 columns]

## Recidivism

I mutate the recidivism ratio (number of habitual offenders over number of admitted convicts) for each state and year.

In [54]:
Recidivism = Recidivism.assign(RecidivRatio= \
                  lambda df: df["habitual_offenders"]/df["convicts_admitted"])
Recidivism

,state_name,year,convicts_admitted,habitual_offenders,RecidivRatio
0,Andhra Pradesh,2001,17345,1063,0.061286
1,Andhra Pradesh,2002,13322,318,0.023870
2,Andhra Pradesh,2003,15682,536,0.034179
3,Andhra Pradesh,2004,14397,399,0.027714
4,Andhra Pradesh,2005,12389,977,0.078860
...,...,...,...,...,...
442,Dadra And Nagar Haveli,2013,0,0,NaN
443,Daman And Diu,2013,9,0,0.000000
444,Delhi,2013,7015,1103,0.157234
445,Lakshadweep,2013,0,0,NaN


In [55]:
pd.merge(Recidivism, Caste, on=["state_name","year"])

C:\Users\Frances\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,state_name,year,convicts_admitted,habitual_offenders,RecidivRatio,"(Female, OBC)","(Female, Others)","(Female, SC)","(Female, ST)","(Male, OBC)","(Male, Others)","(Male, SC)","(Male, ST)"
0,Andhra Pradesh,2001,17345,1063,0.061286,50,12,33,7,1526,939,1236,418
1,Andhra Pradesh,2002,13322,318,0.023870,65,29,34,23,1833,1327,1177,384
2,Andhra Pradesh,2003,15682,536,0.034179,105,31,25,11,2279,1162,1153,381
3,Andhra Pradesh,2004,14397,399,0.027714,116,31,35,15,2146,1204,918,300
4,Andhra Pradesh,2005,12389,977,0.078860,120,19,41,18,2201,1312,1002,298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,Sikkim,2013,125,5,0.040000,1,0,1,0,40,28,8,26
336,Tamil Nadu,2013,3905,29,0.007426,62,38,62,0,1970,779,1390,347
337,Tripura,2013,1485,28,0.018855,5,3,6,0,120,124,104,140
338,Uttar Pradesh,2013,30159,1081,0.035843,380,305,252,26,9260,9045,5593,449
